In [1]:
import pandas as pd
import numpy as np
import os

import psycopg2

In [2]:
file = open("db_login.txt", 'r')
logins = file.readlines()

for i, line in enumerate(logins):
    logins[i] = line.split('\n')[0]
    
file.close()

connect_params = {
     "host"      : logins[0],
     "database"  : logins[1],
     "user"      : logins[2],
     "password"  : logins[3]
}

In [3]:
def connect(conn_params):
    conn = None
    
    try:
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**conn_params)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
        
    print("Connection successful")
    
    return conn

In [4]:
conn = connect(connect_params)

Connecting to the PostgreSQL database...
Connection successful


In [5]:
def postgresql_to_dataframe(conn, select_query, column_headers):
    cursor = conn.cursor()
    
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    tuples = cursor.fetchall()
    cursor.close()
    
    df = pd.DataFrame(tuples, columns=column_headers)
    return df

In [6]:
# All Congressional Candidates
query1 = """
SELECT cycle, cid, first_last_party, party, dist_id_run_for, current_candidate, cycle_candidate, recip_code,
raised_from_pacs, raised_from_individuals, raised_total, raised_unitemized
FROM candidates
WHERE dist_id_run_for NOT LIKE 'PRES' AND dist_id_run_for NOT LIKE '__S_'
"""

In [7]:
col_names_1 = ['cycle', 'id', 'candidate_name', 'party', 'district', 'current_candidate', 'cycle_candidate',
               'recip_code', 'raised_from_pacs', 'raised_from_individuals', 'raised_total', 'raised_unitemized']

In [8]:
# All Senate Candidates
query2 = """
SELECT cycle, cid, first_last_party, party, dist_id_run_for, current_candidate, cycle_candidate, recip_code,
raised_from_pacs, raised_from_individuals, raised_total, raised_unitemized
FROM candidates
WHERE dist_id_run_for NOT LIKE 'PRES' AND dist_id_run_for LIKE '__S_'
"""

In [9]:
col_names_2 = ['cycle', 'id', 'candidate_name', 'party', 'state', 'current_candidate', 'cycle_candidate',
               'recip_code', 'raised_from_pacs', 'raised_from_individuals', 'raised_total', 'raised_unitemized']

In [10]:
all_congr_candids = postgresql_to_dataframe(conn, query1, col_names_1)
all_congr_candids

,cycle,id,candidate_name,party,district,current_candidate,cycle_candidate,recip_code,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized
0,2000,N00005009,Richmond A Soluade Sr (R),R,MO01,,Y,RL,NaN,NaN,0,NaN
1,1996,N00004126,Mark Alan Behnke (R),R,MI07,,,RN,NaN,NaN,0,0.0
2,1996,N00000718,Jim Ford (R),R,NJ10,,,RN,NaN,NaN,0,0.0
3,1996,N00005551,Ernest J Istook (R),R,OK05,Y,Y,RW,130384.0,99600.0,399980,169996.0
4,1996,N00008523,Stephen Wayne Hofman (R),R,MI16,,Y,RL,NaN,5576.0,18050,12474.0
...,...,...,...,...,...,...,...,...,...,...,...,...
51143,2016,N00033316,Joaquin Castro (D),D,TX20,Y,Y,DI,211035.0,188278.0,431521,32208.0
51144,2016,N00025284,Raul M Grijalva (D),D,AZ03,Y,Y,DI,102018.0,41449.0,184909,41442.0
51145,2016,N00034130,Randal Wallace (R),R,SC07,,,RN,1000.0,4800.0,7575,1775.0
51146,2016,N00013846,Jeff Miller (R),R,FL01,Y,Y,RI,319963.0,403622.0,730935,7350.0


In [11]:
all_congr_candids.to_csv('../data/all_congr_candids.csv')

In [12]:
all_sen_candids = postgresql_to_dataframe(conn, query2, col_names_2)
all_sen_candids

,cycle,id,candidate_name,party,state,current_candidate,cycle_candidate,recip_code,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized
0,1996,N00004451,Stephen Bonsal Young (R),R,MNS1,,Y,RL,492.0,97408.0,264254,166354.0
1,1996,N00001670,Raymond J Clatworthy (R),R,DES2,Y,Y,RL,202815.0,618935.0,1636414,814664.0
2,1996,N00000525,Gavin Terence Mills (I),I,VTS1,,,3L,NaN,266.0,266,0.0
3,1996,N00006284,James A McClure (R),R,IDS1,,,RN,NaN,NaN,0,0.0
4,1996,N00000421,Dick Swett (D),D,NHS2,Y,Y,DL,528713.0,1161345.0,3020794,1330736.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10728,1996,N00005453,Mark Brown (D),D,ARS1,,,DL,NaN,720.0,720,NaN
10729,2016,N00037568,Sean Guthrie (I),I,FLS2,Y,Y,3O,NaN,101.0,101,NaN
10730,2016,N00013820,Chris Van Hollen (D),D,MDS2,,,DO,209184.0,4032126.0,4643648,402338.0
10731,2016,N00007836,Maria Cantwell (D),D,WAS1,,,DI,-1750.0,122060.0,302913,182603.0


In [13]:
all_sen_candids.to_csv('../data/all_sen_candids.csv')

In [14]:
conn.close()